In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [4]:
from datasets import load_dataset

reviews = load_dataset("cornell-movie-review-data/rotten_tomatoes")

In [5]:
reviews

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [7]:
reviews["train"].to_pandas()

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
8525,any enjoyment will be hinge from a personal th...,0
8526,if legendary shlockmeister ed wood had ever ma...,0
8527,hardly a nuanced portrait of a young woman's b...,0
8528,"interminably bleak , to say nothing of boring .",0


In [19]:
reviews["test"].to_pandas()

,text,label
0,lovingly photographed in the manner of a golde...,1
1,consistently clever and suspenseful .,1
2,"it's like a "" big chill "" reunion of the baade...",1
3,the story gives ample opportunity for large-sc...,1
4,"red dragon "" never cuts corners .",1
...,...,...
1061,a terrible movie that some people will neverth...,0
1062,there are many definitions of 'time waster' bu...,0
1063,"as it stands , crocodile hunter has the hurrie...",0
1064,the thing looks like a made-for-home-video qui...,0


In [9]:
reviews["validation"].to_pandas()

,text,label
0,compassionately explores the seemingly irrecon...,1
1,the soundtrack alone is worth the price of adm...,1
2,rodriguez does a splendid job of racial profil...,1
3,beneath the film's obvious determination to sh...,1
4,bielinsky is a filmmaker of impressive talent .,1
...,...,...
1061,"this picture is murder by numbers , and as eas...",0
1062,hilarious musical comedy though stymied by acc...,0
1063,"if you are into splatter movies , then you wil...",0
1064,"a dull , simple-minded and stereotypical tale ...",0


In [13]:
# Use a pipeline as a high-level helper
from transformers import pipeline

classification = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [14]:
classification("This movie was great")

[{'label': 'POSITIVE', 'score': 0.999869704246521}]

In [15]:
classification("This movie was worst")

[{'label': 'NEGATIVE', 'score': 0.999795138835907}]

In [16]:
def label(review):
  label = classification(review)[0]['label']
  if label == 'POSITIVE':
    return 1
  else:
    return 0


In [17]:
label("This movie was great")

1

In [18]:
label("This movie was worst")

0

In [21]:
test_df = reviews["test"].to_pandas()

In [28]:
test_df['predicted_label'] = test_df['text'].apply(label)

In [29]:
test_df

,text,label,predicted_label
0,lovingly photographed in the manner of a golde...,1,1
1,consistently clever and suspenseful .,1,1
2,"it's like a "" big chill "" reunion of the baade...",1,0
3,the story gives ample opportunity for large-sc...,1,1
4,"red dragon "" never cuts corners .",1,1
...,...,...,...
1061,a terrible movie that some people will neverth...,0,0
1062,there are many definitions of 'time waster' bu...,0,0
1063,"as it stands , crocodile hunter has the hurrie...",0,0
1064,the thing looks like a made-for-home-video qui...,0,0


In [31]:
test_df['label']==test_df['predicted_label']

,0
0,True
1,True
2,False
3,True
4,True
...,...
1061,True
1062,True
1063,True
1064,True


In [32]:
sum(test_df['label']==test_df['predicted_label'])

956

In [33]:
sum(test_df['label']==test_df['predicted_label'])/1066

0.8968105065666041